# Implementation on Upcoming Race Card - Predict LBW & Odds:

#### Import library's & Data:

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats

from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import metrics
from sklearn.dummy import DummyRegressor

import statsmodels.api as sm

%config InlineBackend.figure_format = 'retina'

plt.style.use('fivethirtyeight')

#### Import Race Card Data:

In [2]:
# Import data: 

df_rc = pd.read_csv('./datasets/race_card.csv', parse_dates=['date'], dayfirst=True)

In [3]:
df_rc.shape


(137, 21)

In [4]:
df_rc.head()

,date,race,class,distance,surface,colour,horse_no,Last 5 Runs,horse_name,EA,...,jockey,trainer,barrier,rating,Wt,Desc,Sire,Dam,Owner,Rating + / -
0,2019-07-21,1,C5,1400,T,NaN,2,95758,MAJESTIC,NaN,...,M ZAKI,D KOK,2,48.0,60,4yo CH USA G NH,MIZZEN MAST (USA),SUNSET RIDE (USA),MAJESTIC STABLE,-1
1,2019-07-21,1,C5,1400,T,NaN,3,0*369,PRINCE PEGASUS,NaN,...,APP M NIZAR,M WALKER,13,48.0,60,5yo B AUS G SH,BENETEAU (AUS),SASSY PEG (AUS),FAIRDEAL STABLE,-1
2,2019-07-21,1,C5,1400,T,NaN,4,53*06,HERO STAR,NaN,...,B MELHAM,D MEAGHER,3,47.0,59.5,5yo B AUS G SH,MAGIC ALBERT (AUS),BELLA CASSANI (AUS),HO LI WEN,-1
3,2019-07-21,1,C5,1400,T,NaN,6,7,COMECATCHME,NaN,...,J AZZOPARDI,ZL MOK,7,46.0,59,3yo B IRE G NH,CAMACHO (GB),FOXTROT PEARL (IRE),FRIENDSHIP STABLE,-
4,2019-07-21,1,C5,1400,T,NaN,7,24*2*,HERO IN THE WIND,NaN,...,S NOH,H TAKAOKA,14,46.0,59,5yo CH JPN G NH,SCREEN HERO (JPN),KIRARIN (JPN),GALAHAD RACING STABLE,1


#### Data wrangling:

In [5]:
df_rc['date'].dt.strftime('%d/%m/%Y')

# cleaing:
df_rc = df_rc.drop(['colour', 'Desc', 'Sire', 'Dam', 'Owner', 'Rating + / -'], axis=1)


# Indexing:
df_rc['indexing'] = np.arange(len(df_rc)) #new indexing column for re-ranking in original order (date & race)
df_rc['surf_numb'] = df_rc.apply(lambda x: 1 if x['surface'] == 'T' else .1, axis=1) 
df_rc['indexing_surf_dist'] = df_rc.surf_numb * df_rc.distance # new column for surface & distance
df_rc.rename(columns={'barrier':'bar'}, inplace=True)
df_rc['indexing_surf_dist_bar'] = df_rc.indexing_surf_dist * df_rc.bar # new column for suf; dist & bar
df_rc['indexing_date_race'] = df_rc["date"].map(str) + df_rc["race"].map(str) # new column for date & race

df_rc['indexing_surf_dist_10'] = df_rc['indexing_surf_dist'].astype(str)
df_rc['indexing_surf_dist_horse'] = df_rc["indexing_surf_dist_10"
                                         ].map(str) + df_rc["horse_name"].map(str) # new column

#df_rc.sub['jockey']('[-0-9]+', '', mystring)


In [6]:
df_rc.shape

(137, 22)

#### Import Engineered Data:

In [7]:
# Import data: 
df = pd.read_csv('./datasets/stc_data_engineered.csv', parse_dates=['date'], dayfirst=True)
df = df.drop(['Unnamed: 0'], axis=1)

#df.head()

#### Merge Engineered Featuresinto df_rc:

In [8]:
# Feature 1:
# Groupby horse_name, then calc max to temp_1
temp_1 = df.groupby(['horse_name']).horse_median_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_median_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1 = df_rc.groupby(['indexing_date_race']).horse_median_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_median_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df 'f2_horse_lengths_vs_max_in_race' calc
df_rc = df_rc.assign(feature_1 = ((df_rc['horse_median_vs_standard'] - 
                                             df_rc['max_race_horse_median_vs_standard'] ) )) 



In [9]:
#df.loc[['BIG ELEPHANT']].head()
#df.loc[(df['horse_name'] == 'STORMY VIEW') ].T

In [10]:
# Feature 1a:
# Groupby horse_name, then calc max to temp_1a
temp_1a = df.groupby(['horse_name']).horse_mean_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1a, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_mean_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1a = df_rc.groupby(['indexing_date_race']).horse_mean_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1a, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_mean_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_1a = ((df_rc['horse_mean_vs_standard'] - 
                                             df_rc['max_race_horse_mean_vs_standard'] ) )) 



In [11]:
# Feature 1b:
# Groupby horse_name, then calc max to temp_1b
temp_1b = df.groupby(['horse_name']).horse_top_qtr_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1b, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_top_qtr_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1b = df_rc.groupby(['indexing_date_race']).horse_top_qtr_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1b, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_top_qtr_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_1b = ((df_rc['horse_top_qtr_vs_standard'] - 
                                             df_rc['max_race_horse_top_qtr_vs_standard'] ) )) 



In [12]:
# Feature 1c:
# Groupby horse_name, then calc max to temp_1b
temp_1c = df.groupby(['horse_name']).horse_max_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1c, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_max_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1c = df_rc.groupby(['indexing_date_race']).horse_max_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1c, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_max_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_1c = ((df_rc['horse_max_vs_standard'] - 
                                             df_rc['max_race_horse_max_vs_standard'] ) )) 



In [13]:
# Feature 1d:
# Groupby horse_name, then calc max to temp_1d
temp_1d = df.groupby(['horse_name']).horse_min_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1d, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_min_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1d = df_rc.groupby(['indexing_date_race']).horse_min_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1d, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_min_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_1d = ((df_rc['horse_min_vs_standard'] - 
                                             df_rc['max_race_horse_min_vs_standard'] ) )) 



In [14]:
# Feature 1e:
# Groupby horse_name, then calc max to temp_1e
temp_1e = df.groupby(['horse_name']).horse_bot_qtr_vs_standard.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1e, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_bot_qtr_vs_standard'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_1e = df_rc.groupby(['indexing_date_race']).horse_bot_qtr_vs_standard.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_1e, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_bot_qtr_vs_standard'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_1e = ((df_rc['horse_bot_qtr_vs_standard'] - 
                                             df_rc['max_race_horse_bot_qtr_vs_standard'] ) )) 



In [15]:
# Feature 3:
# Groupby horse_name, then calc max 
temp_3 = df.groupby(['horse_name']).horse_median_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_median_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3 = df_rc.groupby(['indexing_date_race']).horse_median_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_median_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column 
df_rc = df_rc.assign(feature_3 = ((df_rc['horse_median_vs_standard_3'] - 
                                             df_rc['max_race_horse_median_vs_standard_3'] ) )) 



In [16]:
# Feature 3a:
# Groupby horse_name, then calc max 
temp_3a = df.groupby(['horse_name']).horse_mean_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3a, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_mean_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3a = df_rc.groupby(['indexing_date_race']).horse_mean_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3a, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_mean_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_3a = ((df_rc['horse_mean_vs_standard_3'] - 
                                             df_rc['max_race_horse_mean_vs_standard_3'] ) )) 



In [17]:
# Feature 3b:
# Groupby horse_name, then calc max 
temp_3b = df.groupby(['horse_name']).horse_top_qtr_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3b, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_top_qtr_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3b = df_rc.groupby(['indexing_date_race']).horse_top_qtr_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3b, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_top_qtr_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_3b = ((df_rc['horse_top_qtr_vs_standard_3'] - 
                                             df_rc['max_race_horse_top_qtr_vs_standard_3'] ) )) 



In [18]:
## Feature 3c:
# Groupby horse_name, then calc max 
temp_3c = df.groupby(['horse_name']).horse_max_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3c, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_max_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3c = df_rc.groupby(['indexing_date_race']).horse_max_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3c, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_max_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_3c = ((df_rc['horse_max_vs_standard_3'] - 
                                             df_rc['max_race_horse_max_vs_standard_3'] ) )) 



In [19]:
# Feature 3d:
# Groupby horse_name, then calc max 
temp_3d = df.groupby(['horse_name']).horse_min_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3d, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_min_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3d = df_rc.groupby(['indexing_date_race']).horse_min_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3d, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_min_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_3d = ((df_rc['horse_min_vs_standard_3'] - 
                                             df_rc['max_race_horse_min_vs_standard_3'] ) )) 



In [20]:
# Feature 3e:
# Groupby horse_name, then calc max 
temp_3e = df.groupby(['horse_name']).horse_bot_qtr_vs_standard_3.agg(['max'])

# merge back into df_rc groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3e, on='horse_name', how='left').fillna(-3) # dummy number for now
df_rc.rename(columns={'max': 'horse_bot_qtr_vs_standard_3'}, inplace=True)


# MAX IN RACE OF (HORSE MAX SPEED FIGURE (LENGTHS VS STANDARD))
# Groupby date & surf, then calc max for each race as save in temp df_rc:
temp_3e = df_rc.groupby(['indexing_date_race']).horse_bot_qtr_vs_standard_3.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_3e, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_race_horse_bot_qtr_vs_standard_3'}, inplace=True)


# HORSE FIG - RACE MAX 
# new column in df_rc
df_rc = df_rc.assign(feature_3e = ((df_rc['horse_bot_qtr_vs_standard_3'] - 
                                             df_rc['max_race_horse_bot_qtr_vs_standard_3'] ) )) 



In [21]:
# Jockey Win %
# Groupby horse_name calc median to temp_4
temp_5 = df.groupby(['jockey']).jock_pct.agg(['mean'])

# merge median avg lengths vs standard back into df groupby horse (this is the horses speed fig)
df_rc = pd.merge(df_rc, temp_5, on='jockey', how='left').fillna(0) # dummy for now
df_rc.rename(columns={'mean': 'jock_pct'}, inplace=True)

# Groupby date & race, then sum jockey % (to be used in calc of each jockey's race % relative):
temp_5 = df_rc.groupby(['indexing_date_race']).jock_pct.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, temp_5, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_jock_pct_race'}, inplace=True)

# new column in df 'f5_jock_pct_race' calc (sort of expressed as lbw)
df_rc = df_rc.assign(jock_pct_race_f5 = 
               ((df_rc['jock_pct'] - df_rc['max_jock_pct_race'] ) )) 


In [22]:
#temp_5.loc[(temp_5['jock_pct'] == 'JP VAN DER MERWE') ]
#df.loc[(df['indexing_date_race'] ) ]
#df.loc[(df['horse_name'] == 'PEGASUS JUNIOR') ]
#df_rc.loc[(df_rc['jockey'] == 'APP WH KOK') ]

#df_rc[['indexing_date_race', 'jockey', 'jock_pct']].head(15)

#df_rc[['date', 'race', 'horse_name', 'jockey', 'jock_pct', 'max_jock_pct_race', 'jock_pct_race_f5']].tail(300)

In [23]:
# Trainer Win %
# Groupby horse_name calc mean
temp_6 = df.groupby(['trainer']).trainer_pct.agg(['mean'])

# merge 
df_rc = pd.merge(df_rc, temp_6, on='trainer', how='left').fillna(0) # dummy number for now
df_rc.rename(columns={'mean': 'trainer_pct'}, inplace=True)

# Groupby date & race, 
temp_6 = df_rc.groupby(['indexing_date_race']).trainer_pct.agg(['max'])

# merge max back 
df_rc = pd.merge(df_rc, temp_6, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_trainer_pct_race'}, inplace=True)

# new column 
df_rc = df_rc.assign(trainer_pct_race_f6 = 
               ((df_rc['trainer_pct'] - df_rc['max_trainer_pct_race'] ) )) 


In [24]:
# Trainer Jockey Combo Win %
# new column 'jock_trainer_pct':
df_rc = df_rc.assign(jock_trainer_combo = (df_rc['trainer'] + df_rc['jockey']) )

# Groupby horse_name calc mean
temp_7 = df.groupby(['jock_trainer_combo']).jock_trainer_win_pct.agg(['mean'])

# merge 
df_rc = pd.merge(df_rc, temp_7, on='jock_trainer_combo', how='left').fillna(0) # dummy number for now
df_rc.rename(columns={'mean': 'jock_trainer_win_pct'}, inplace=True)

# Groupby date & race, 
temp_7 = df_rc.groupby(['indexing_date_race']).jock_trainer_win_pct.agg(['max'])

# merge max back 
df_rc = pd.merge(df_rc, temp_7, on='indexing_date_race')
df_rc.rename(columns={'max': 'max_jock_trainer_pct_race'}, inplace=True)

# new column 
df_rc = df_rc.assign(jock_trainer_win_pct_race_f7 = 
               ((df_rc['jock_trainer_win_pct'] - df_rc['max_jock_trainer_pct_race'] ) )) 


In [25]:
# Barrier %
# Groupby horse_name calc mean
temp_8 = df.groupby(['indexing_surf_dist_bar']).bar_win_mean.agg(['mean'])

# merge 
df_rc = pd.merge(df_rc, temp_8, on='indexing_surf_dist_bar', how='left').fillna(0) # dummy number for now
df_rc.rename(columns={'mean': 'bar_win_mean'}, inplace=True)

# Groupby date & race, 
temp_8 = df_rc.groupby(['indexing_date_race']).bar_win_mean.agg(['max'])

# merge max back 
df_rc = pd.merge(df_rc, temp_8, on='indexing_date_race')
df_rc.rename(columns={'max': 'bar_win_max_race'}, inplace=True)

# new column 
df_rc = df_rc.assign(bar_win_race_f8 = 
               ((df_rc['bar_win_mean'] - df_rc['bar_win_max_race'] ) )) 


In [26]:
# return df to original order:
df_rc = df_rc.sort_values(by=['indexing'], ascending =True)
df_rc = df_rc.reset_index()
df_rc = df_rc.drop('index', axis=1)

In [27]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_rc.tail(12)

,date,race,class,distance,surface,horse_no,Last 5 Runs,horse_name,EA,gear,jockey,trainer,bar,rating,Wt,indexing,surf_numb,indexing_surf_dist,indexing_surf_dist_bar,indexing_date_race,indexing_surf_dist_10,indexing_surf_dist_horse,horse_median_vs_standard,max_race_horse_median_vs_standard,feature_1,horse_mean_vs_standard,max_race_horse_mean_vs_standard,feature_1a,horse_top_qtr_vs_standard,max_race_horse_top_qtr_vs_standard,feature_1b,horse_max_vs_standard,max_race_horse_max_vs_standard,feature_1c,horse_min_vs_standard,max_race_horse_min_vs_standard,feature_1d,horse_bot_qtr_vs_standard,max_race_horse_bot_qtr_vs_standard,feature_1e,horse_median_vs_standard_3,max_race_horse_median_vs_standard_3,feature_3,horse_mean_vs_standard_3,max_race_horse_mean_vs_standard_3,feature_3a,horse_top_qtr_vs_standard_3,max_race_horse_top_qtr_vs_standard_3,feature_3b,horse_max_vs_standard_3,max_race_horse_max_vs_standard_3,feature_3c,horse_min_vs_standard_3,max_race_horse_min_vs_standard_3,feature_3d,horse_bot_qtr_vs_standard_3,max_race_horse_bot_qtr_vs_standard_3,feature_3e,jock_pct,max_jock_pct_race,jock_pct_race_f5,trainer_pct,max_trainer_pct_race,trainer_pct_race_f6,jock_trainer_combo,jock_trainer_win_pct,max_jock_trainer_pct_race,jock_trainer_win_pct_race_f7,bar_win_mean,bar_win_max_race,bar_win_race_f8
125,2019-07-21,10,C4,1400,T,3,51371,BILLY BRITAIN,-3,-3,APP I AMIRUL,S GRAY,10,64.0,58.5,125,1,1400,14000,2019-07-21 00:00:0010,1400,1400BILLY BRITAIN,2.214444,5.341234,-3.126790,1.523725,5.252694,-3.728969,6.250000,7.102738,-0.852738,7.547743,14.690252,-7.142510,-6.197262,0.802738,-7.000000,-2.346023,4.747743,-7.093766,5.919145,11.142114,-5.222969,3.231233,12.850637,-9.619404,9.017043,17.533933,-8.516889,16.603846,48.179983,-31.576136,-11.607886,3.792114,-15.400000,-3.390160,7.642114,-11.032274,8.880309,11.875,-2.994691,8.146640,13.250518,-5.103878,S GRAYAPP I AMIRUL,3.719008,8.947368,-5.228360,6.918239,12.162162,-5.243923
126,2019-07-21,10,C4,1400,T,4,7*25,WIND OF LIBERTY,-3,EM,M LERNER,H TAKAOKA,6,64.0,58.5,126,1,1400,8400,2019-07-21 00:00:0010,1400,1400WIND OF LIBERTY,3.602738,5.341234,-1.738496,2.902738,5.252694,-2.349955,5.877738,7.102738,-1.225000,6.402738,14.690252,-8.287514,-1.997262,0.802738,-2.800000,0.627738,4.747743,-4.120005,2.742114,11.142114,-8.400000,3.354614,12.850637,-9.496023,4.667114,17.533933,-12.866819,9.392114,48.179983,-38.787869,-1.457886,3.792114,-5.250000,1.429614,7.642114,-6.212500,8.673469,11.875,-3.201531,7.783019,13.250518,-5.467499,H TAKAOKAM LERNER,3.500000,8.947368,-5.447368,9.602649,12.162162,-2.559513
127,2019-07-21,10,C4,1400,T,5,1231*,AXEL,-3,VIS,R WOODWORTH,M WALKER,14,62.0,57.5,127,1,1400,19600,2019-07-21 00:00:0010,1400,1400AXEL,2.943735,5.341234,-2.397499,2.489587,5.252694,-2.763107,4.361889,7.102738,-2.740849,9.170907,14.690252,-5.519345,-4.056265,0.802738,-4.859003,0.192216,4.747743,-4.555527,7.685622,11.142114,-3.456492,6.250662,12.850637,-6.599974,10.121070,17.533933,-7.412862,16.882244,48.179983,-31.297739,-8.412449,3.792114,-12.204563,0.849745,7.642114,-6.792369,8.968610,11.875,-2.906390,9.079284,13.250518,-4.171234,M WALKERR WOODWORTH,8.947368,8.947368,0.000000,11.111111,12.162162,-1.051051
128,2019-07-21,10,C4,1400,T,6,41689,DRONE,-3,-3,J AZZOPARDI,M CLEMENTS,8,62.0,57.5,128,1,1400,11200,2019-07-21 00:00:0010,1400,1400DRONE,2.090252,5.341234,-3.250982,-1.252187,5.252694,-6.504880,3.981562,7.102738,-3.121177,7.070907,14.690252,-7.619345,-14.709748,0.802738,-15.512486,-6.852523,4.747743,-11.600266,-5.264378,11.142114,-16.406492,-3.189005,12.850637,-16.039641,7.681113,17.533933,-9.852820,25.632244,48.179983,-22.547739,-31.207886,3.792114,-35.000000,-9.464378,7.642114,-17.106492,11.235955,11.875,-0.639045,13.250518,13.250518,0.000000,M CLEMENTSJ AZZOPARDI,4.285714,8.947368,-4.661654,8.675799,12.162162,-3.486363
129,2019-07-21,10,C4,1400,T,7,1*164,IRVING LIPSCHITZ,-3,B,D MOOR,DL FREEDMAN,2,60.0,56.5,129,1,1400,2800,2019-07-21 00:00:0010,1400,1400IRVING LIPSCHITZ,5.018033,5.341234,-0.323202,5.252694,5.

# Predict Race Finishing Position 'race_lbw_pred' back to the df:

In [28]:
# create a Python list of feature names
X_trainfinal = df[['feature_1', 'feature_1a', 'feature_1b', 'feature_1c', 'feature_1d', 'feature_1e',
                   'feature_3', 'feature_3a', 'feature_3b', 'feature_3c', 'feature_3d', 'feature_3e',
                   'jock_pct_race_f5', 'trainer_pct_race_f6', 'jock_trainer_win_pct_race_f7', 'bar_win_race_f8'
                    ]]
y_trainfinal = df['lbw']


X_testfinal = df_rc[['feature_1', 'feature_1a', 'feature_1b', 'feature_1c', 'feature_1d', 'feature_1e',
                   'feature_3', 'feature_3a', 'feature_3b', 'feature_3c', 'feature_3d', 'feature_3e',
                   'jock_pct_race_f5', 'trainer_pct_race_f6', 'jock_trainer_win_pct_race_f7', 'bar_win_race_f8'
                    ]]
#y_testfinal = df_rc['lbw']

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_trainfinal, y_trainfinal)

# make predictions on the testing set
y_predfinal = linreg.predict(X_testfinal)

In [29]:
y_predfinal.shape

(137,)

In [30]:
y_predfinal

array([ -2.95244011,  -2.20557505,  -2.58417963, -12.6238329 ,
        -2.48160821,  -2.02350213,  -4.76119383,  -7.16715073,
        -6.63706394,  -6.7545239 ,  -1.98696381,  -5.92752333,
        -6.59502072,  -6.54107005,  -0.45565357,  -2.95225849,
        -8.17413487,  -9.9340932 ,  -5.19905283,  -8.26062952,
        -7.04790639, -10.55500862,  -6.97749955,  -7.23511854,
        -6.50133075, -20.83272759, -13.8400162 ,  -7.17517703,
        -3.07602634, -14.31337439,  -3.04569088,  -3.5997879 ,
        -1.24766387,  -2.56612562,  -1.2508756 ,  -1.87578469,
        -4.72127396,  -9.2995821 ,  -6.23220228,  -5.1241691 ,
        -5.84145288,  -4.34230922,  -3.36636876,  -3.19303954,
        -3.07144759,  -3.44654273,  -2.59896056, -11.65787326,
        -3.49781841,  -3.9601089 ,  -3.61665131,  -3.89735018,
        -6.30193511,  -5.99735142,  -3.69329732,  -5.57817394,
        -2.97531464,  -2.30353319,  -6.22018323,  -4.87451143,
        -5.68728977,  -5.81519916,  -8.1740634 ,  -2.44

In [31]:
# assign y_pred_total back to lbw_pred in df:
df_rc = df_rc.assign(lbw_pred = y_predfinal)

# Groupby date & race then (max_lbw) merge back to df for adj lbw_pred best back to 0:
race_lbw_max = df_rc.groupby(['indexing_date_race']).lbw_pred.agg(['max'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, race_lbw_max, on='indexing_date_race')
df_rc.rename(columns={'max': 'race_lbw_max'}, inplace=True)

# return df to original order:
df_rc = df_rc.sort_values(by=['indexing'], ascending =True)
df_rc = df_rc.reset_index()
df_rc = df_rc.drop('index', axis=1)

# new column in df 'race_lbw_pred' calc
df_rc = df_rc.assign(race_lbw_pred = ((df_rc['lbw_pred'] - df_rc['race_lbw_max'] ) )) 

In [32]:
df_rc[['date', 'race', 'horse_name', 'race_lbw_pred']].head(14)

,date,race,horse_name,race_lbw_pred
0,2019-07-21,1,MAJESTIC,-0.965476
1,2019-07-21,1,PRINCE PEGASUS,-0.218611
2,2019-07-21,1,HERO STAR,-0.597216
3,2019-07-21,1,COMECATCHME,-10.636869
4,2019-07-21,1,HERO IN THE WIND,-0.494644
5,2019-07-21,1,PLATO,-0.036538
6,2019-07-21,1,SILVER JOY,-2.774230
7,2019-07-21,1,YULONG FIRE,-5.180187
8,2019-07-21,1,ZMAN,-4.650100
9,2019-07-21,1,BROADWAY SUCCESS,-4.767560


In [33]:
# return df to original order:
df = df.sort_values(by=['indexing'], ascending =True)
df = df.reset_index()
df = df.drop('index', axis=1)

# Assign Odds

In [34]:
# assign prob_pred from race_lbw_pred based on past data relationship to df
df_rc['prob_pred'] = [80 if x > -0.05 
                   else 75 if x > -0.1
                   else 70 if x > -0.15
                   else 65 if x > -0.2
                   else 60 if x > -0.25
                   else 55 if x > -0.3
                   else 50 if x > -0.55
                   else 45 if x > -0.8 
                   else 40 if x > -1.05
                   else 35 if x > -1.3 
                   else 30 if x > -1.8 
                   else 25 if x > -2.3 
                   else 20 if x > -2.85
                   else 15 if x > -3.4
                   else 10 if x > -4.75
                   else  5 if x > -6.1
                   else 1
                   for x in df_rc['race_lbw_pred']]

In [35]:
# Groupby date & race then (race_prob_pred_sum) merge back to df for adj prob_pred to race_prob_pred:
race_prob_pred_sum = df_rc.groupby(['indexing_date_race']).prob_pred.agg(['sum'])

# merge max back into df groupby indexing_date_race:
df_rc = pd.merge(df_rc, race_prob_pred_sum, on='indexing_date_race')
df_rc.rename(columns={'sum': 'race_prob_pred_sum'}, inplace=True)

# new column in df 'race_prob_pred' calc
df_rc = df_rc.assign(race_prob_pred = ((df_rc['prob_pred'] / df_rc['race_prob_pred_sum'] ) * 100 )) 

# new column in df 'race_win_div_pred' calc
df_rc = df_rc.assign(race_win_div_pred = ((5 / ((df_rc['race_prob_pred']) / 100)) )) 

In [36]:
#df_rc.loc[(df_rc['date'] == '2019-07-07') & (df_rc['race'] == 2)].T

In [37]:
#df.loc[(df['date'] == '2019-07-07') & (df['race'] == 2)]

In [38]:
#df_rc.loc[(df_rc['horse_name'] == 'STORMY VIEW')]

In [39]:
#df.loc[(df['horse_name'] == 'STORMY VIEW')]

# Predicted LBW & Odds

In [40]:
# Top picks & Fair-odds:

df_rc_bets = df_rc[['date', 'race', 'horse_name', 'jockey', 'horse_no', 
                    'race_lbw_pred', 'race_win_div_pred']]

#df_rc_bets.round({'races_run': 0, "race_lbw_pred":1, "race_prob_pred":1, "race_win_div_pred":0}) 

df_rc_bets.sort_values(['date', 'race','race_win_div_pred'],ascending=True).groupby('race').head(1).round(1)



,date,race,horse_name,jockey,horse_no,race_lbw_pred,race_win_div_pred
5,2019-07-21,1,PLATO,APP WH KOK,8,-0.0,26.6
14,2019-07-21,2,ON LINE,V DURIC,1,0.0,7.8
32,2019-07-21,3,DUSSELDORF,M RODD,5,0.0,22.6
46,2019-07-21,4,GALILEO'S APPROACH,M KELLADY,5,0.0,30.1
57,2019-07-21,5,RUNMINDERBINDERRUN,APP WH KOK,2,0.0,24.1
74,2019-07-21,6,RICHEBOURG,R WOODWORTH,5,0.0,13.9
87,2019-07-21,7,CHALAZA,APP WH KOK,5,0.0,20.4
102,2019-07-21,8,EAGLE EYE,D MOOR,7,0.0,20.3
111,2019-07-21,9,QUARTER BACK,V DURIC,5,0.0,26.6
129,2019-07-21,10,IRVING LIPSCHITZ,D MOOR,7,0.0,16.5


In [41]:

df_rc_bets.loc[df_rc.race == 10].round(1)


,date,race,horse_name,jockey,horse_no,race_lbw_pred,race_win_div_pred
123,2019-07-21,10,GOLDEN FLAME,B THOMPSON,1,-1.1,37.7
124,2019-07-21,10,ABSOLVIDO,APP WH KOK,2,-0.8,29.3
125,2019-07-21,10,BILLY BRITAIN,APP I AMIRUL,3,-3.9,132.0
126,2019-07-21,10,WIND OF LIBERTY,M LERNER,4,-2.4,66.0
127,2019-07-21,10,AXEL,R WOODWORTH,5,-2.3,66.0
128,2019-07-21,10,DRONE,J AZZOPARDI,6,-6.7,1320.0
129,2019-07-21,10,IRVING LIPSCHITZ,D MOOR,7,0.0,16.5
130,2019-07-21,10,SIAM SAPPHIRE,G BOSS,8,-2.0,52.8
131,2019-07-21,10,O'WHAT A FEELING,M KELLADY,9,-3.7,132.0
132,2019-07-21,10,OXBOW SUN,CC WONG,10,-5.8,264.0


In [42]:
df_rc.loc[df_rc.race == 9].round(1)

,date,race,class,distance,surface,horse_no,Last 5 Runs,horse_name,EA,gear,jockey,trainer,bar,rating,Wt,indexing,surf_numb,indexing_surf_dist,indexing_surf_dist_bar,indexing_date_race,indexing_surf_dist_10,indexing_surf_dist_horse,horse_median_vs_standard,max_race_horse_median_vs_standard,feature_1,horse_mean_vs_standard,max_race_horse_mean_vs_standard,feature_1a,horse_top_qtr_vs_standard,max_race_horse_top_qtr_vs_standard,feature_1b,horse_max_vs_standard,max_race_horse_max_vs_standard,feature_1c,horse_min_vs_standard,max_race_horse_min_vs_standard,feature_1d,horse_bot_qtr_vs_standard,max_race_horse_bot_qtr_vs_standard,feature_1e,horse_median_vs_standard_3,max_race_horse_median_vs_standard_3,feature_3,horse_mean_vs_standard_3,max_race_horse_mean_vs_standard_3,feature_3a,horse_top_qtr_vs_standard_3,max_race_horse_top_qtr_vs_standard_3,feature_3b,horse_max_vs_standard_3,max_race_horse_max_vs_standard_3,feature_3c,horse_min_vs_standard_3,max_race_horse_min_vs_standard_3,feature_3d,horse_bot_qtr_vs_standard_3,max_race_horse_bot_qtr_vs_standard_3,feature_3e,jock_pct,max_jock_pct_race,jock_pct_race_f5,trainer_pct,max_trainer_pct_race,trainer_pct_race_f6,jock_trainer_combo,jock_trainer_win_pct,max_jock_trainer_pct_race,jock_trainer_win_pct_race_f7,bar_win_mean,bar_win_max_race,bar_win_race_f8,lbw_pred,race_lbw_max,race_lbw_pred,prob_pred,race_prob_pred_sum,race_prob_pred,race_win_div_pred
107,2019-07-21,9,G1,1800,T,1,0,ELITE QUARTETO,-3,BB,S NOH,C BROWN,11,101.0,57,107,1,1800,19800,2019-07-21 00:00:009,1800,1800ELITE QUARTETO,1.5,8.5,-7.0,0.3,7.0,-6.7,2.9,9.3,-6.4,4.3,16.3,-12.0,-4.8,1.5,-6.3,-1.7,7.1,-8.8,-4.0,12.5,-16.5,-5.8,10.5,-16.3,0.2,16.3,-16.1,4.5,24.1,-19.6,-18.0,2.3,-20.3,-11.0,7.7,-18.7,16.8,18.1,-1.3,13.8,13.8,0.0,C BROWNS NOH,0.0,13.1,-13.1,15.8,19.2,-3.4,-8.4,-2.8,-5.6,5,425,1.2,425.0
108,2019-07-21,9,G1,1800,T,2,1*175,NIMBLE,-3,-3,J POWELL,L DRAGON,7,91.0,57,108,1,1800,12600,2019-07-21 00:00:009,1800,1800NIMBLE,7.2,8.5,-1.3,4.8,7.0,-2.2,8.1,9.3,-1.2,12.0,16.3,-4.2,-15.3,1.5,-16.8,5.5,7.1,-1.6,7.0,12.5,-5.5,0.3,10.5,-10.2,12.7,16.3,-3.6,15.0,24.1,-9.1,-58.3,2.3,-60.5,3.5,7.7,-4.3,14.3,18.1,-3.8,9.5,13.8,-4.3,L DRAGONJ POWELL,11.8,13.1,-1.3,6.3,19.2,-12.9,-5.8,-2.8,-3.0,15,425,3.5,141.7
109,2019-07-21,9,G1,1800,T,3,50808,KING LOUIS,-3,-3,W PIKE,R LE GRANGE,14,85.0,57,109,1,1800,25200,2019-07-21 00:00:009,1800,1800KING LOUIS,5.7,8.5,-2.8,6.3,7.0,-0.8,9.2,9.3,-0.1,11.4,16.3,-4.9,0.8,1.5,-0.7,4.2,7.1,-2.9,12.5,12.5,0.0,8.5,10.5,-2.0,13.7,16.3,-2.6,22.3,24.1,-1.8,-5.8,2.3,-8.1,2.0,7.7,-5.8,0.0,18.1,-18.1,11.8,13.8,-2.0,R LE GRANGEW PIKE,0.0,13.1,-13.1,10.0,19.2,-9.2,-3.8,-2.8,-1.0,40,425,9.4,53.1
110,2019-07-21,9,G1,1800,T,4,90000,MR CLINT,-3,"B, TT",B PINHEIRO,DL FREEDMAN,15,85.0,57,110,1,1800,27000,2019-07-21 00:00:009,1800,1800MR CLINT,5.7,8.5,-2.8,6.0,7.0,-1.0,8.2,9.3,-1.1,11.3,16.3,-5.0,0.1,1.5,-1.4,4.3,7.1,-2.8,6.3,12.5,-6.2,7.8,10.5,-2.7,10.2,16.3,-6.1,19.4,24.1,-4.7,1.2,2.3,-1.1,4.6,7.7,-3.1,0.0,18.1,-18.1,12.6,13.8,-1.2,DL FREEDMANB PINHEIRO,0.0,13.1,-13.1,0.0,19.2,-19.2,-5.0,-2.8,-2.2,25,425,5.9,85.0
111,2019-07-21,9,G1,1800,T,5,11176,QUARTER BACK,-3,WK,V DURIC,M CLEMENTS,4,85.0,57,111,1,1800,7200,2019-07-21 00:00:009,1800,1800QUARTER BACK,8.5,8.5,0.0,6.7,7.0,-0.4,9.3,9.3,0.0,11.4,16.3,-4.9,-10.4,1.5,-11.9,7.1,7.1,0.0,11.6,12.5,-0.9,5.2,10.5,-5.3,13.9,16.3,-2.4,17.5,24.1,-6.7,-32.4,2.3,-34.7,2.8,7.7,-4.9,18.1,18.1,0.0,13.3,13.8,-0.5,M CLEMENTSV DURIC,13.1,13.1,0.0,15.6,19.2,-3.6,-2.8,-2.8,0.0,80,425,18.8,26.6
112,2019-07-21,9,G1,1800,T,6,11122,SACRED CROIX,-3,"TT, VIS",R WOODWORTH,M WALKER,8,85.0,57,112,1,1800,14400,2019-07-21 00:00:009,1800,1800SACRED CROIX,4.2,8.5,-4.2,4.6,7.0,-2.4,8.5,9.3,-0.8,11.3,16.3,-5.0,-2.0,1.5,-3.5,0.9,7.1,-6.2,9.9,12.5,-2.6,10.5,10.5,0.0,16.3,16.3,0.0,19.2,24.1,-4.9,-1.2,2.3,-3.5,6.2,7.7,-1.6,9.0,18.1,-9.1,9.1,13.8,-4.7,M WALKERR WOODWORTH,8.9,13.1,-4.1,5.7,19.2,-13.5,-4.5,-2.8,-1.7,30,425,7.1,70.8
113,2019-07-21,9,G1,1800,T,7,11160,CIRCUIT MISSION,-3,-3,M RODD,DL FREEDMAN,1,83.0,57,113,

In [43]:
# save df_rc dataset back to folder

df_rc_bets.to_csv('./datasets/stc_data_race_card_analyzed.csv')

In [44]:
# Odds in Horse Order
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#df_rc[['date', 'race', 'horse_name', 'feature_1', 'feature_1a', 'feature_1b', 'feature_1c', 'feature_1d',
#       'feature_1e',
#      ]].head(123)


In [45]:
# Odds in Horse Order
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#df_rc[['date', 'race', 'horse_name', 'feature_3', 'feature_3a', 'feature_3b', 'feature_3c', 'feature_3d',
#       'feature_3e'
#      ]].head(123)


In [46]:
# Odds in Horse Order
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#df_rc[['date', 'race', 'horse_name', 'jock_pct_race_f5', 'trainer_pct_race_f6', 
#       'jock_trainer_win_pct_race_f7', 'bar_win_race_f8', 'race_win_div_pred'
#      ]].head(123)


In [47]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#df[['date', 'race', 'horse_name', 'jock_pct_race_f5', 'trainer_pct_race_f6',
#    'jock_trainer_win_pct_race_f7', 'bar_win_race_f8', 'win_div_3'
#      ]].tail(208)

In [48]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#df.loc[(df['date'] == '2019-01-01') & (df['race'] == 2)].T

###  https://datatofish.com/multiple-linear-regression-python/